___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Регуляризация в SciKit-Learn

Ранее мы создали набор полиномиальных признаков и затем применили для них линейную регрессию. Но мы можем сделать модель ещё лучше, используя регуляризацию.

Регуляризация старается минимизировать метрику "сумма квадратов остатков" (RSS - residual sum of squares) *и* "функцию штрафа". Функция штрафа старается "наказывать" модели с очень большими коэффициентами. Некоторые методы регуляризации по сути приводят к тому, что некоторые неполезные признаки получают нулевые коэффициенты, то есть модель не будет использовать эти признаки.

Давайте рассмотрим два метода регуляризации - гребневая регрессия (или ридж-регрессия, или регрессия риджа, англ. ridge regression) и регрессия лассо. Мы будем использовать их в сочетании с полиномиальными признаками, поскольку регуляризация модели не так эффективна на исходном наборе признаков набора данных X.

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка данных и подготовительные шаги

In [2]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

### Полиномиальная конвертация

In [3]:
from sklearn.preprocessing import PolynomialFeatures

In [4]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [5]:
poly_features = polynomial_converter.fit_transform(X)

### Разбиение на обучающий и тестовый наборы данных -  Train | Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

----
----

## Масштабирование данных

Хотя в наших данных все признаки имеют один и тот же порядок величины (в $1000 долларов), это не всегда верно в других наборах данных. Поскольку математика регуляризации будет суммировать коэффициенты, очень важно стандартизировать признаки. Посмотрите теоретические видео-лекции для более подробного рассказа на эту тему, а также почему мы делаем **обучение (fit)** только на обучающих данных, и делаем **трансформацию** обоих наборов данных отдельно друг от друга.

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# help(StandardScaler)

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

StandardScaler()

In [12]:
X_train = scaler.transform(X_train)

In [13]:
X_test = scaler.transform(X_test)

## Гребневая регрессия (Ridge Regression)

Посмотрите видео-лекции по этой теме - там мы рассказываем про гребневую регрессию и про выбор параметра alpha.

In [14]:
from sklearn.linear_model import Ridge

In [15]:
ridge_model = Ridge(alpha=10)

In [16]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [17]:
test_predictions = ridge_model.predict(X_test)

In [18]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [19]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [20]:
MAE

0.5774404204714173

In [21]:
RMSE

0.8946386461319676

Какова точность на обучающей выборке? (Мы будем использовать это позже для сравнения)

In [22]:
# Производительность на обучающем наборе
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.5288348183025331

### Выбираем значение alpha с помощью кросс-валидации

Детали есть в видео-лекции.

In [23]:
from sklearn.linear_model import RidgeCV

In [24]:
# help(RidgeCV)

In [25]:
# Выбираем способ оценки модели: https://scikit-learn.org/stable/modules/model_evaluation.html
# Отрицательные значения RMSE. Все метрики следуют принципу "больше значит лучше".

# Смотрим все варианты: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [26]:
# Чем больше значений alpha Вы укажете, тем дольше времени потребуется.
# К счастью наш набор данных достаточно маленький.
ridge_cv_model.fit(X_train,y_train)

RidgeCV(scoring='neg_mean_absolute_error')

In [27]:
ridge_cv_model.alpha_

0.1

In [28]:
test_predictions = ridge_cv_model.predict(X_test)

In [29]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [30]:
MAE

0.4273774884348172

In [31]:
RMSE

0.6180719926963456

In [32]:
# Метрика MAE на обучающем наборе
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.30941321056368337

In [41]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [35]:
poly_features.shape

(200, 19)


-----

## Регрессия Лассо

In [36]:
from sklearn.linear_model import LassoCV

In [58]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1,n_alphas=1000,cv=5)

In [59]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1, n_alphas=1000)

In [60]:
lasso_cv_model.alpha_

0.49430709092258285

In [38]:
test_predictions = lasso_cv_model.predict(X_test)

In [39]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [40]:
MAE

0.6541723161252868

In [41]:
RMSE

1.1308001022762548

In [42]:
# Метрика MAE на обучающем наборе
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.691280714082071

In [43]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

Elastic Net сочетает в себе ридж-регрессию и лассо-регрессию, чтобы взять лучшее из обеих регуляризаций!

In [61]:
from sklearn.linear_model import ElasticNetCV

In [63]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [64]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [65]:
elastic_model.l1_ratio_

1.0

In [66]:
test_predictions = elastic_model.predict(X_test)

In [49]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [50]:
MAE

0.566326211756945

In [51]:
RMSE

0.7485546215633726

In [52]:
# Метрика MAE на обучающем наборе
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.4307582990472368

In [53]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

In [56]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [57]:
from sklearn.preprocessing import PolynomialFeatures

In [58]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [59]:
poly_features = polynomial_converter.fit_transform(X)

In [60]:
poly_features.shape

(200, 19)

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=30, random_state=101)

In [63]:
X_train.shape

(170, 19)

In [64]:
from sklearn.preprocessing import StandardScaler

In [65]:
scaler = StandardScaler()

In [66]:
scaler.fit(X_train)

StandardScaler()

In [67]:
X_train = scaler.transform(X_train)

In [68]:
X_test = scaler.transform(X_test)

In [69]:
X_train[0]

array([-1.44576589,  0.83224632,  1.5903232 , -1.11520745, -0.79196689,
       -0.62793152,  0.69033939,  1.6268041 ,  1.51047793, -0.8964398 ,
       -0.78845386, -0.68317495, -0.5711305 , -0.37291184, -0.3153736 ,
        0.49645432,  1.35976586,  1.52043282,  1.15511982])

In [70]:
poly_features[0]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

In [1]:
from sklearn.linear_model import Ridge

In [72]:
ridge_model = Ridge(alpha=10)

In [73]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [74]:
test_predictions = ridge_model.predict(X_test)

In [75]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [76]:
MAE = mean_absolute_error(y_test, test_predictions)

In [77]:
MAE

0.46791031329382565

In [78]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [79]:
RMSE

0.5800973421167421

In [2]:
from sklearn.linear_model import RidgeCV

In [81]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')

In [82]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(scoring='neg_mean_absolute_error')

In [83]:
ridge_cv_model.alpha_

0.1

In [84]:
from sklearn.metrics._scorer import _SCORERS

In [85]:
_SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'd2_absolute_error_score', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_w

In [86]:
test_predictions = ridge_cv_model.predict(X_test)

In [87]:
MAE = mean_absolute_error(y_test, test_predictions)

In [88]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [89]:
MAE

0.3897222129994162

In [90]:
RMSE

0.5358695098747467

In [91]:
ridge_cv_model.coef_

array([ 6.21462058,  0.35503221,  0.55543803, -7.61288572,  4.39892388,
       -1.01358175, -0.41255052,  0.25373593, -0.33430282,  3.2886831 ,
       -0.99004616,  0.61362846,  0.37421681, -0.25130418,  0.24458708,
        0.21001722, -0.16901378,  0.15200217, -0.04148747])

In [92]:
ridge_cv_model.best_score_

-0.3666881026518571

In [3]:
from sklearn.linear_model import LassoCV

In [110]:
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, cv=5, max_iter=100000)

In [111]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, max_iter=100000)

In [112]:
lasso_cv_model.alpha_

0.004927481100593532

In [113]:
test_predictions = lasso_cv_model.predict(X_test)

In [114]:
test_predictions

array([14.27748113, 19.32406791, 12.3518185 , 16.99693483,  7.95825836,
        6.77125217, 20.15637701, 17.28730419, 10.4398255 , 19.88147914,
        9.67428653, 13.92248798, 11.83879397, 23.21163391, 19.39770607,
        9.25120813, 12.23432388,  9.29864071,  8.48995747, 21.46064032,
        6.91075171, 19.26867304, 27.39559892, 24.49414787,  9.13204338,
       11.56706017, 20.93201244,  9.26295506, 12.55723973,  8.44838995])

In [115]:
MAE = mean_absolute_error(y_test, test_predictions)

In [116]:
MAE

0.38124893476835325

In [117]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [118]:
RMSE

0.51073408149606

In [119]:
lasso_cv_model.coef_

array([ 5.69712982e+00,  2.67092548e-01,  2.49370077e-01, -6.38053625e+00,
        3.92210950e+00, -2.46900079e-01,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  2.45113501e+00, -3.09117262e-01, -0.00000000e+00,
        4.10713994e-02, -0.00000000e+00, -0.00000000e+00,  6.34934869e-02,
        0.00000000e+00, -0.00000000e+00, -8.75181329e-04])

In [122]:
from sklearn.linear_model import ElasticNetCV

In [126]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps=0.001, n_alphas=100, max_iter=1000000)

In [127]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [129]:
elastic_model.l1_ratio_

1.0

In [130]:
elastic_model.alpha_

0.004927481100593532

In [132]:
lasso_cv_model.alpha_

0.004927481100593532

In [139]:
MAE = mean_absolute_error(y_test, test_predictions)

In [140]:
MAE

0.38124893476835325